### Intro to Federated Learning (Example)

In [1]:
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy

hook = sy.TorchHook(torch)

In [2]:
# Create virtual workers

toby = sy.VirtualWorker(hook, id="toby")
julie = sy.VirtualWorker(hook, id="julie")

In [3]:
# Fake dataset
data = torch.tensor([[3,1],[2,6],[2,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data, send some to toby/julie
toby_data = data[0:2]
toby_target = target[0:2]

julie_data = data[2:]
julie_target = target[2:]

In [4]:
# lets look at the data!
toby_data

tensor([[3., 1.],
        [2., 6.]], grad_fn=<SliceBackward>)

In [5]:
toby_target

tensor([[0.],
        [0.]], grad_fn=<SliceBackward>)

In [6]:
julie_data

tensor([[2., 0.],
        [1., 1.]], grad_fn=<SliceBackward>)

In [7]:
julie_target

tensor([[1.],
        [1.]], grad_fn=<SliceBackward>)

In [8]:
# Initialize fake model
model = nn.Linear(2,1)
toby_data = toby_data.send(toby)
julie_data = julie_data.send(julie)
toby_target = toby_target.send(toby)
julie_target = julie_target.send(julie)

In [9]:
# examine what everythng looks like
toby_data.location._objects

{30786630407: tensor([[3., 1.],
         [2., 6.]], requires_grad=True),
 77150670350: tensor([[0.],
         [0.]], requires_grad=True)}

In [10]:
julie_data

(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647]

In [11]:
toby_target

(Wrapper)>[PointerTensor | me:36849338081 -> toby:77150670350]

In [12]:
julie_target

(Wrapper)>[PointerTensor | me:75798397008 -> julie:6533946021]

In [13]:
# organize pointers into list
datasets = [[toby_data, toby_target], [julie_data, julie_target]]
datasets

[[(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407],
  (Wrapper)>[PointerTensor | me:36849338081 -> toby:77150670350]],
 [(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647],
  (Wrapper)>[PointerTensor | me:75798397008 -> julie:6533946021]]]

In [24]:
def train():
    # Training logic stuff
    for iter in range(10):

        # go through each worker data
        for data,target in datasets:
            
            print (data) # want to see what things look like
            # send model to the worker
            model.send(data.location)
            
            opt = optim.get_optim(data.location.id) # on the Optims object

            # ensure you get a clean gradient
            opt.zero_grad()

            pred = model(data)

            # calculate loss
            loss = ((pred - target)**2).sum()

            # identify weights that caused loss
            loss.backward()

            # update weights
            opt.step()
            
            model.get()

            # print progress
            print(loss.get().data) 


In [25]:
train()

(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407]
tensor(1.8191)
(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647]
tensor(0.6406)
(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407]
tensor(1.8191)
(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647]
tensor(0.6406)
(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407]
tensor(1.8191)
(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647]
tensor(0.6406)
(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407]
tensor(1.8191)
(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647]
tensor(0.6406)
(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407]
tensor(1.8191)
(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647]
tensor(0.6406)
(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407]
tensor(1.8191)
(Wrapper)>[PointerTensor | me:52001299336 -> julie:22736638647]
tensor(0.6406)
(Wrapper)>[PointerTensor | me:1767604664 -> toby:30786630407]
te